# Daum Storts 페이지에서 경기 정보 크롤링 #

In [5]:
# 필요한 모듈 임포트
import time                                                             # 크롤링시 서버 부하를 줄이기 위한 지연 시간 설정
from selenium import webdriver                                          # Selenium을 사용하여 동적 웹 페이지를 처리하기 위한 모듈
from selenium.webdriver.chrome.options import Options as ChromeOptions  # ChromeOptions를 사용하여 브라우저 설정을 조정
from selenium.webdriver.common.by import By                             # 요소를 찾기 위한 By 모듈
from selenium.webdriver.support.ui import WebDriverWait                 # WebDriverWait를 사용하여 특정 조건을 기다림
from selenium.webdriver.support import expected_conditions as EC        # expected_conditions를 사용하여 특정 조건을 정의
import pandas as pd                                                     # pandas를 사용하여 데이터프레임을 처리하고 CSV 파일로 저장                                                      
from io import StringIO                                                 # 문자열을 파일처럼 처리하기 위한 모듈

# selenium에서 사용될 options 설정
chrome_options = ChromeOptions()
chrome_options.add_argument("--headless")               # 헤드리스 모드로 실행 (브라우저 창을 띄우지 않음)
chrome_options.add_argument("--disable-gpu")            # GPU 가속 비활성화
chrome_options.add_argument("--no-sandbox")             # 샌드박스 모드 비활성화, 샌드박스 모드는 일부 환경에서 문제가 될 수 있음
chrome_options.add_argument("--window-size=1920x1080")  # 브라우저 창 크기 설정
driver = webdriver.Chrome(options=chrome_options)       # ChromeDriver 설정
#list_sports = ["wvl"]
list_sports = ["kl", "kbo", "vl", "kbl", "wkbl"]
list_years = ["2023", "2024"]
list_months = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"] 

for sports in list_sports:
    sports_df_for_current_sport = None # 각 스포츠마다 DataFrame을 초기화
    print(f"===== {sports.upper()} 리그 데이터 수집 시작 =====")
    for year in list_years:
        print(f"--- {year}년 데이터 수집 시작 ---")
        for month in list_months:
            url = f"https://sports.daum.net/schedule/{sports}?date={year}{month}"
            print(f"\n현재 URL: {url}")
            html_content = "" # 각 URL 요청 전에 초기화
            try:
                print(f"Selenium으로 '{url}' 페이지 여는 중...")
                driver.get(url)
                print("페이지의 동적 컨텐츠가 로드되기를 기다리는 중... (최대 20초)")
                wait = WebDriverWait(driver, 20)

                wait.until(EC.presence_of_element_located((By.XPATH, "//table[contains(@class, 'tbl_sched')]//tbody/tr")))
                print("컨텐츠가 로드되었습니다.")

                html_content = driver.page_source
            except Exception as e:
                print(f"Selenium 실행 중 오류 발생: {e}")
                print("ChromeDriver가 올바르게 설정되었는지, 인터넷 연결이 원활한지 확인해주세요.")
                continue # 다음 월/연도/스포츠로 넘어감

            if html_content:
                try:
                    list_of_dataframes = pd.read_html(StringIO(html_content), encoding='utf-8')
                    if not list_of_dataframes or len(list_of_dataframes) < 2: # 테이블이 없거나 예상보다 적을 경우
                        print(f"Selenium으로 가져온 HTML에서 스케줄 테이블을 찾을 수 없습니다. (URL: {url})")
                    else:
                        # 보통 두 번째 테이블이지만, 페이지 구조에 따라 다를 수 있으므로 확인 필요
                        schedule_df = list_of_dataframes[1]
                        print(f"\n선택된 테이블 컬럼: {schedule_df.columns.tolist()}")

                        if not schedule_df.empty:
                            # 날짜 형식 변환 및 연도 추가 (월별 데이터프레임에 바로 적용)
                            # '날짜' 컬럼이 'MM.DD(요일)' 형식이므로 앞에 연도를 붙여줌
                            schedule_df['날짜'] = f"{year}." + schedule_df['날짜'].astype(str)

                            print("선택된 테이블 데이터 일부 (연도 추가 후):")
                            print(schedule_df.head())

                            if sports_df_for_current_sport is not None:
                                sports_df_for_current_sport = pd.concat([sports_df_for_current_sport, schedule_df], ignore_index=True)
                            else:
                                sports_df_for_current_sport = schedule_df.copy() # 첫 데이터는 복사
                        else:
                            print(f"\n선택된 테이블에 데이터가 없습니다. (URL: {url})")
                            print("페이지에 실제로 해당 날짜의 데이터가 없는지, 또는 다른 문제가 있는지 확인해보세요.")
                except ValueError as e:
                    print(f"Selenium으로 가져온 HTML을 파싱하는 중 오류 발생 (pandas): {e} (URL: {url})")
                except IndexError as e:
                    print(f"테이블 인덱싱 오류: 스케줄 테이블을 찾을 수 없습니다. list_of_dataframes[1] 접근 실패. (URL: {url})")
                except Exception as e:
                    print(f"Selenium 데이터 처리 중 알 수 없는 오류 발생: {e} (URL: {url})")
            else:
                print("\nSelenium을 통해 웹 페이지 내용을 가져오지 못했습니다. 이전 오류 메시지를 확인해주세요.")
            
            time.sleep(1) # 서버 부하를 줄이기 위해 약간의 지연 추가 (선택 사항)

    if sports_df_for_current_sport is not None and not sports_df_for_current_sport.empty:
        output_csv_file = f"{sports}_schedule_{list_years[0]}_{list_years[-1]}.csv" # 파일명에 연도 범위 포함
        # '리그'와 '중계/기록' 컬럼이 없을 수도 있으므로 errors='ignore' 사용
        sports_df_for_current_sport.drop(columns=['리그', '중계/기록'], inplace=True, errors='ignore')
        sports_df_for_current_sport.to_csv(output_csv_file, index=False, encoding='utf-8-sig')
        print(f"\n'{output_csv_file}' 파일로 성공적으로 저장되었습니다!")
        print("최종 DataFrame 일부:")
        print(sports_df_for_current_sport.head())
        print(sports_df_for_current_sport.tail())
    else:
        print(f"\n{sports.upper()} 리그에 대한 데이터를 수집하지 못했거나 데이터가 비어있어 CSV 파일을 생성하지 않습니다.")

if 'driver' in locals() and driver is not None:
    driver.quit()
    print("\nWebDriver 종료됨.")

===== KL 리그 데이터 수집 시작 =====
--- 2023년 데이터 수집 시작 ---

현재 URL: https://sports.daum.net/schedule/kl?date=202301
Selenium으로 'https://sports.daum.net/schedule/kl?date=202301' 페이지 여는 중...
페이지의 동적 컨텐츠가 로드되기를 기다리는 중... (최대 20초)
컨텐츠가 로드되었습니다.

선택된 테이블 컬럼: ['날짜', '시간', '구장', '경기  홈팀원정팀', '리그', '구분', '중계/기록']
선택된 테이블 데이터 일부 (연도 추가 후):
             날짜         시간         구장  경기  홈팀원정팀         리그         구분  \
0  2023.01.01 일  경기가 없습니다.  경기가 없습니다.  경기가 없습니다.  경기가 없습니다.  경기가 없습니다.   
1  2023.01.02 월  경기가 없습니다.  경기가 없습니다.  경기가 없습니다.  경기가 없습니다.  경기가 없습니다.   
2  2023.01.03 화  경기가 없습니다.  경기가 없습니다.  경기가 없습니다.  경기가 없습니다.  경기가 없습니다.   
3  2023.01.04 수  경기가 없습니다.  경기가 없습니다.  경기가 없습니다.  경기가 없습니다.  경기가 없습니다.   
4  2023.01.05 목  경기가 없습니다.  경기가 없습니다.  경기가 없습니다.  경기가 없습니다.  경기가 없습니다.   

       중계/기록  
0  경기가 없습니다.  
1  경기가 없습니다.  
2  경기가 없습니다.  
3  경기가 없습니다.  
4  경기가 없습니다.  

현재 URL: https://sports.daum.net/schedule/kl?date=202302
Selenium으로 'https://sports.daum.net/schedule/kl?date=202302' 페이지 여는 중...
페이지의 동적 컨

# Daum Sports 페이지에서 크롤링한 데이터 중 필요없는 정보 삭제, 필요한 정보 생성, 컬럼 정리

In [7]:
input_csv_file_list = ['wkbl_schedule_2023_2024', 'kbl_schedule_2023_2024', 'kbo_schedule_2023_2024', 'kl_schedule_2023_2024', 'vl_schedule_2023_2024', 'wvl_schedule_2023_2024']
output_filename_suffix = '_transformed'
for input_csv_file in input_csv_file_list:
    output_csv_file = input_csv_file + output_filename_suffix + '.csv'
    input_csv_file += '.csv'
    print(f"\n파일 처리 중...: {input_csv_file} -> {output_csv_file}")
    try:
        df = pd.read_csv(input_csv_file)
        # "시간" 컬럼에 "경기가 없습니다." 라는 문자열이 있다면 해당 행 전체를 삭제
        df_cleaned = df[df['시간'] != "경기가 없습니다."].copy() # Use .copy() to avoid SettingWithCopyWarning

        # "날짜" 컬럼 변경 및 "요일" 컬럼 추가
        if not df_cleaned.empty:
            # Example: "2023.01.01 일" -> ["2023.01.01", "일"]
            split_date_day = df_cleaned['날짜'].str.split(' ', n=1, expand=True)
            df_cleaned['날짜'] = split_date_day[0]
            df_cleaned['요일'] = split_date_day[1]
        else:
            df_cleaned['요일'] = pd.Series(dtype='object')


        # 3. "경기  홈팀원정팀" 컬럼을 "홈팀","홈팀점수", "원정팀","원정팀점수" 로 컬럼 재구성
        game_info_col = '경기  홈팀원정팀'
        if input_csv_file in 'kbo_schedule_2023_2024.csv':
            game_info_col = '경기원정팀홈팀'

        if not df_cleaned.empty and game_info_col in df_cleaned.columns:
            processed_game_info = df_cleaned[game_info_col].str.replace('종료  ', '', regex=False, n=1)

            if input_csv_file in 'kbl_schedule_2023_2024.csv':
                game_details = processed_game_info.str.split(r'\s+', n=5, expand=True)
                df_cleaned['홈팀'] = game_details[0] + ' ' + game_details[1]
                df_cleaned['홈팀점수'] = game_details[2]
                df_cleaned['원정팀'] = game_details[3] + ' ' + game_details[4]
                df_cleaned['원정팀점수'] = game_details[5]
            else:
                game_details = processed_game_info.str.split(r'\s+', n=3, expand=True)    
                df_cleaned['홈팀'] = game_details[0]
                df_cleaned['홈팀점수'] = game_details[1]
                df_cleaned['원정팀'] = game_details[2]
                df_cleaned['원정팀점수'] = game_details[3]
            
            df_cleaned = df_cleaned.drop(columns=[game_info_col])

            df_cleaned['홈팀승리여부'] = df_cleaned['홈팀점수'].astype(str) > df_cleaned['원정팀점수'].astype(str)
            
        elif game_info_col not in df_cleaned.columns:
            print(f"\nColumn '{game_info_col}' not found. Skipping game info reconstruction.")
        else:
            df_cleaned['홈팀'] = pd.Series(dtype='object')
            df_cleaned['홈팀점수'] = pd.Series(dtype='object')
            df_cleaned['원정팀'] = pd.Series(dtype='object')
            df_cleaned['원정팀점수'] = pd.Series(dtype='object')

        desired_columns = [
            '날짜', '요일', '시간', '구장',
            '홈팀', '홈팀점수', '원정팀', '원정팀점수', '홈팀승리여부',
            '리그', '구분', '중계/기록'
        ]
        
        final_df_columns = [col for col in desired_columns if col in df_cleaned.columns]
        df_transformed = df_cleaned[final_df_columns]

        df_transformed.to_csv(output_csv_file, index=False, encoding='utf-8-sig')
        print(f"\nTransformed 데이터를 csv로 저장했습니다. '{output_csv_file}'")

    except FileNotFoundError:
        print(f"Error: '{input_csv_file}' 파일이 없습니다.")
    except KeyError as e:
        print(f"Error: CSV에서 필요한 컬럼을 찾지 못했습니다.: {e}")
        print("CSV 파일의 컬럼명을 확인해주세요.")
    except Exception as e:
        print(f"예기치 못한 에러가 발생했습니다.: {e}")


파일 처리 중...: wkbl_schedule_2023_2024.csv -> wkbl_schedule_2023_2024_transformed.csv

Transformed 데이터를 csv로 저장했습니다. 'wkbl_schedule_2023_2024_transformed.csv'

파일 처리 중...: kbl_schedule_2023_2024.csv -> kbl_schedule_2023_2024_transformed.csv

Transformed 데이터를 csv로 저장했습니다. 'kbl_schedule_2023_2024_transformed.csv'

파일 처리 중...: kbo_schedule_2023_2024.csv -> kbo_schedule_2023_2024_transformed.csv

Transformed 데이터를 csv로 저장했습니다. 'kbo_schedule_2023_2024_transformed.csv'

파일 처리 중...: kl_schedule_2023_2024.csv -> kl_schedule_2023_2024_transformed.csv

Transformed 데이터를 csv로 저장했습니다. 'kl_schedule_2023_2024_transformed.csv'

파일 처리 중...: vl_schedule_2023_2024.csv -> vl_schedule_2023_2024_transformed.csv

Transformed 데이터를 csv로 저장했습니다. 'vl_schedule_2023_2024_transformed.csv'

파일 처리 중...: wvl_schedule_2023_2024.csv -> wvl_schedule_2023_2024_transformed.csv

Transformed 데이터를 csv로 저장했습니다. 'wvl_schedule_2023_2024_transformed.csv'


# 배구 공홈에서 관중수가 포함된 경기 가져오기 (https://kovo.co.kr/) #
- 경기가 남자, 여자로 구분되어 있지만 남녀구분없이 모든 데이터를 가져오는 것으로 함.



In [8]:
# 정보를 가져올 URL 예시
# https://api.kovo.co.kr/api/game/summary?season=0{season}&gPart={cat}&gNum={match}
# season = ["19", "20", "21", "22", "23"]
# cat = ["201", "202", "203", "204"]
# match = 1 ~ 269

In [ ]:
import requests
import pandas as pd
import time
import os

base_url = "https://api.kovo.co.kr/api/game/summary"
all_game_records = []
seasons = [f"{s:03}" for s in range(19, 22)]  # 019, 020, 021
gparts_to_try = [f"{gp:03}" for gp in range(201, 205)] # 201, 202, 203, 204

for season in seasons:
    print(f"\nProcessing Season: {season}")
    for gNum in range(1, 270):  # gNum from 1 to 269
        found_data_for_gnum = False
        print(f"  Processing gNum: {gNum:03}", end="")
        
        for gPart in gparts_to_try:
            params = {
                "season": season,
                "gPart": gPart,
                "gNum": gNum
            }
            try:
                #print(f"    Trying gPart: {gPart} with URL: {requests.Request('GET', base_url, params=params).prepare().url}")
                response = requests.get(base_url, params=params, timeout=10)
                response.raise_for_status()  # Raise an exception for HTTP errors (4xx or 5xx)
                
                data = response.json()
                
                if data["result"]:
                    print(f" - Success with gPart: {gPart}")
                    game_summary = data["result"] 
                    game_summary['fetch_season'] = season
                    game_summary['fetch_gNum'] = gNum
                    game_summary['fetch_gPart_success'] = gPart
                    all_game_records.append(game_summary)
                    found_data_for_gnum = True
                    break
                else:
                    pass

            except requests.exceptions.Timeout:
                print(f"시간초과: season={season}, gPart={gPart}, gNum={gNum}")
            except requests.exceptions.HTTPError as e:
                print(f"HTTP 오류 발생: season={season}, gPart={gPart}, gNum={gNum}: {e}")
            except requests.exceptions.RequestException as e:
                print(f"요청에러: season={season}, gPart={gPart}, gNum={gNum}: {e}")
            except ValueError as e:
                print(f"Json 해석 에러: season={season}, gPart={gPart}, gNum={gNum}. Response: {response.text[:200]}")
            
            time.sleep(0.1)

        if not found_data_for_gnum:
            print(f"크롤링할 데이터 없음: season {season}, gNum {gNum:03} after trying all gParts.")

df = None
if not all_game_records:
    print("\n CSV로 저장할 데이터를 가져오지 못했습니다.")
else:
    df = pd.DataFrame(all_game_records)

if df is not None and not df.empty:
    output_filename = "kovo_game_summaries.csv"
    try:
        df.to_csv(output_filename, index=False, encoding='utf-8-sig')
        print(f"\nCSV 파일로 저장 했습니다. {os.path.abspath(output_filename)}")
    except Exception as e:
        print(f"\nCSV 파일 생성에 실패했습니다. : {e}")
else:
    print("\n데이터 프레임에 데이터가 없습니다.")


Processing Season: 019
  Processing gNum: 200 - Success with gPart: 201
  Processing gNum: 201 - Success with gPart: 201
  Processing gNum: 202 - Success with gPart: 201
  Processing gNum: 203 - Success with gPart: 201
  Processing gNum: 204 - Success with gPart: 201
  Processing gNum: 205 - Success with gPart: 201
  Processing gNum: 206 - Success with gPart: 201
  Processing gNum: 207 - Success with gPart: 201
  Processing gNum: 208 - Success with gPart: 201
  Processing gNum: 209 - Success with gPart: 201
  Processing gNum: 210 - Success with gPart: 201
  Processing gNum: 211 - Success with gPart: 201
  Processing gNum: 212 - Success with gPart: 201
  Processing gNum: 213 - Success with gPart: 201
  Processing gNum: 214 - Success with gPart: 201
  Processing gNum: 215 - Success with gPart: 201
  Processing gNum: 216 - Success with gPart: 201
  Processing gNum: 217 - Success with gPart: 201
  Processing gNum: 218 - Success with gPart: 201
  Processing gNum: 219 - Success with gPart: 